# Multiprocessing in Python: Unlocking True Parallelism
https://docs.python.org/3/library/multiprocessing.html

These notes cover how to use Python's `multiprocessing` module to perform parallel processing, effectively bypassing the Global Interpreter Lock (GIL) to achieve significant performance gains for CPU-bound tasks.

## 1. Why Multiprocessing? The Global Interpreter Lock (GIL)

At the heart of CPython is the **Global Interpreter Lock (GIL)**, a mutex that protects access to Python objects, preventing multiple native threads from executing Python bytecodes at the same time within a single process.

- **Multithreading (`threading` module):** Excellent for I/O-bound tasks (e.g., network requests, file operations). While one thread waits for I/O, the GIL is released, allowing another thread to run. However, for CPU-bound tasks (e.g., complex calculations, data processing), threads don't offer true parallelism; they just take turns executing on a single CPU core.
- **Multiprocessing (`multiprocessing` module):** The solution for CPU-bound tasks. This module creates separate processes, each with its own Python interpreter and memory space. Since each process has its own GIL, they can run in parallel on different CPU cores, achieving true concurrency. 🚀

The key takeaway is:
- **I/O-bound task?** Use **multithreading**.
- **CPU-bound task?** Use **multiprocessing**.

## 2. The `multiprocessing` Module: Core Concepts

The `multiprocessing` module's API is intentionally similar to the `threading` module's, making it easy to transition.

#### Creating and Running Processes

The fundamental object is the `multiprocessing.Process` class.

**Key Methods:**
- `p = Process(target=func, args=(arg1, arg2))`: Instantiates a process. `target` is the callable object (function) to be invoked by the `run()` method. `args` is the tuple of arguments for the target.
- `p.start()`: Starts the process's activity. It arranges for the object’s `run()` method to be invoked in a separate process.
- `p.join()`: Blocks the main program until the process whose `join()` method is called terminates. This is crucial for ensuring processes complete before the main script exits.
- `p.is_alive()`: Returns `True` if the process is still running.
- `os.getpid()`: Returns the current process ID.

**Example: Basic Process Creation**

In [1]:
import multiprocessing
import os
import time

def worker_function(name):
    """A simple function that a process will execute."""
    print(f"Worker '{name}' started. Process ID: {os.getpid()}")
    time.sleep(2)
    print(f"Worker '{name}' finished.")

# On Windows, this guard is essential!
# In a Jupyter cell, this check doesn't behave the same way as a .py script.
# For demonstration, we'll run it directly, but remember to use the guard in standalone scripts.

print(f"Main process ID: {os.getpid()}")

# Create a process
p = multiprocessing.Process(target=worker_function, args=('Alice',))

# Start the process
p.start()

print("Main program continues to run while worker is executing...")

# Wait for the process to complete
p.join()

print("Worker process has finished. Main program exiting.")

Main process ID: 18228
Main program continues to run while worker is executing...
Worker process has finished. Main program exiting.


#### The `if __name__ == '__main__':` Guard

On Windows (and macOS), the `multiprocessing` module uses the **'spawn'** start method by default. This means a new child Python interpreter is started for each new process. This child interpreter re-imports the script. Without the `if __name__ == '__main__':` guard, the process creation code would run again in the child process, leading to an infinite loop of process creation and a `RuntimeError`.

**Therefore, on Windows, all multiprocessing code in `.py` scripts must be placed inside this block.** Jupyter notebooks handle this context differently, so you may not always see an error, but it is a critical best practice when writing standalone applications.

## 3. Inter-Process Communication (IPC) and Synchronization

Since processes have separate memory spaces, they cannot directly share data like threads. We need explicit IPC mechanisms.

#### `multiprocessing.Queue`

A **process-safe and thread-safe** FIFO queue. It's the most common and robust way to pass messages between processes. Data is pickled before being sent and unpickled upon receipt.

- `q.put(obj)`: Puts an object onto the queue.
- `q.get()`: Removes and returns an object from the queue. Blocks until an item is available.

**Example: Using a Queue**

In [2]:
import multiprocessing
import time

def producer(q):
    """Puts items into the queue."""
    for i in range(5):
        item = f"Item {i}"
        q.put(item)
        print(f"Produced: {item}")
        time.sleep(0.5)
    q.put(None) # Sentinel value to signal completion

def consumer(q):
    """Gets items from the queue."""
    while True:
        item = q.get()
        if item is None: # Check for sentinel
            break
        print(f"Consumed: {item}")

queue = multiprocessing.Queue()

p1 = multiprocessing.Process(target=producer, args=(queue,))
p2 = multiprocessing.Process(target=consumer, args=(queue,))

p1.start()
p2.start()

p1.join()
p2.join()

print("All tasks finished.")

All tasks finished.


#### `multiprocessing.Pipe`

A `Pipe()` returns a pair of connection objects connected by a pipe which by default is duplex (two-way). Pipes are generally faster than Queues but are limited to communication between two processes.

- `parent_conn, child_conn = Pipe()`: Creates the pipe.
- `conn.send(obj)`: Sends an object.
- `conn.recv()`: Receives an object.

#### Shared Memory: `Value` and `Array`

For sharing simple data types, you can use `Value` and `Array`, which provide shared memory mapped objects. These should always be protected with a `Lock` to prevent race conditions.

- `num = multiprocessing.Value('d', 0.0)`: A shared double, initialized to 0.0.
- `arr = multiprocessing.Array('i', range(10))`: A shared array of integers.
- `lock = multiprocessing.Lock()`: Creates a lock object.

In [ ]:
import multiprocessing
import os
import time

# --- Example 1: Using multiprocessing.Pipe for Two-Way Communication ---

def pipe_child_process(conn):
    """
    This function runs in the child process.
    It receives data from the parent, processes it, and sends a response back.
    """
    print(f"\n[Pipe Child, PID: {os.getpid()}] Waiting to receive a message...")
    
    # conn.recv() will block until a message is available on the pipe
    message_from_parent = conn.recv()
    print(f"[Pipe Child, PID: {os.getpid()}] Received: '{message_from_parent}'")
    
    # Process the data (e.g., make it uppercase)
    processed_message = message_from_parent.upper()
    
    # Send the processed data back to the parent
    conn.send(processed_message)
    print(f"[Pipe Child, PID: {os.getpid()}] Sent response: '{processed_message}'")
    
    # Close the connection end in the child process
    conn.close()

def run_pipe_example():
    """
    Demonstrates two-way communication between a parent and child process
    using a Pipe.
    """
    print("--- Starting Multiprocessing Pipe Example ---")
    
    # 1. Create a Pipe. It returns two connection objects.
    # By default, the pipe is duplex (two-way).
    parent_conn, child_conn = multiprocessing.Pipe()

    # 2. Create a Process, giving it one end of the pipe (child_conn)
    p = multiprocessing.Process(target=pipe_child_process, args=(child_conn,))
    p.start()

    print(f"[Pipe Parent, PID: {os.getpid()}] Sending message: 'hello from parent'")
    # 3. Parent sends an object through its end of the pipe (parent_conn)
    parent_conn.send("hello from parent")

    # 4. Parent waits to receive the response from the child
    response = parent_conn.recv()
    print(f"[Pipe Parent, PID: {os.getpid()}] Received response: '{response}'")

    # 5. Wait for the child process to finish
    p.join()

    # Close the parent's end of the pipe
    parent_conn.close()
    print("\n--- Pipe Example Finished ---")


# --- Example 2: Using Shared Memory (Value and Array) with a Lock ---

def shared_memory_worker(shared_value, shared_array, lock):
    """
    This function is executed by multiple processes.
    It uses a lock to safely modify a shared counter (Value) and a shared Array.
    """
    process_id = os.getpid()
    for _ in range(5):
        # The 'with' statement automatically acquires and releases the lock.
        # This prevents race conditions.
        with lock:
            # Safely increment the shared value
            shared_value.value += 1
            
            # Safely modify the shared array
            # Here, we increment each element of the array
            for i in range(len(shared_array)):
                shared_array[i] += 1
        
        print(f"[Shared Mem Worker, PID: {process_id}] Incremented. Current value: {shared_value.value}")
        time.sleep(0.01) # Small delay to make execution order more apparent

def run_shared_memory_example():
    """
    Demonstrates sharing state between processes using Value and Array,
    synchronized with a Lock to prevent race conditions.
    """
    print("\n\n--- Starting Shared Memory (Value, Array, Lock) Example ---")
    
    # 1. Create a Lock to synchronize access to shared resources
    lock = multiprocessing.Lock()
    
    # 2. Create shared memory objects
    # 'd' is a type code for a double-precision float.
    # 'i' is a type code for a signed integer.
    shared_counter = multiprocessing.Value('i', 0)
    initial_array = [10, 20, 30, 40]
    shared_array = multiprocessing.Array('i', initial_array)
    
    print(f"Initial shared counter: {shared_counter.value}")
    # Slicing is needed to print the array's contents nicely
    print(f"Initial shared array: {shared_array[:]}")
    print("-" * 20)

    # 3. Create a pool of worker processes
    num_processes = 4
    processes = []
    for _ in range(num_processes):
        p = multiprocessing.Process(
            target=shared_memory_worker, 
            args=(shared_counter, shared_array, lock)
        )
        processes.append(p)
        p.start()

    # 4. Wait for all processes to complete their execution
    for p in processes:
        p.join()
        
    print("-" * 20)
    print("Final Results:")
    print(f"Final shared counter: {shared_counter.value}")
    print(f"Final shared array: {shared_array[:]}")
    print("\n--- Shared Memory Example Finished ---")


if __name__ == '__main__':
    # On Windows, all multiprocessing code must be inside this block
    # to prevent issues with child processes re-importing and re-executing the script.
    
    # Run the first example for Pipe
    run_pipe_example()
    
    # Run the second example for shared memory
    run_shared_memory_example()


--- Starting Multiprocessing Pipe Example ---
[Pipe Parent, PID: 18228] Sending message: 'hello from parent'


**Example: Shared Counter with a Lock**

In [3]:
import multiprocessing

def increment(shared_value, lock):
    for _ in range(10000):
        with lock:
            shared_value.value += 1

shared_counter = multiprocessing.Value('i', 0)
lock = multiprocessing.Lock()

processes = [multiprocessing.Process(target=increment, args=(shared_counter, lock)) for _ in range(4)]

for p in processes:
    p.start()

for p in processes:
    p.join()

print(f"Final counter value: {shared_counter.value}") # Should be 40000

Final counter value: 0


## 4. Managing Pools of Workers: `multiprocessing.Pool`

Creating and managing individual processes can be tedious. The `Pool` object offers a convenient way to parallelize the execution of a function across multiple input values, distributing the input data across a fixed number of processes (a "pool").

`pool = multiprocessing.Pool(processes=os.cpu_count())`

#### Key Pool Methods

- **Blocking (Synchronous) Methods:** These wait for all results to be completed.
    - `pool.map(func, iterable)`: Applies `func` to each item in `iterable`. It chunks the iterable and distributes it to the worker processes. Returns a list of results. It's like a parallel version of the built-in `map()`.
    - `pool.apply(func, args)`: Calls `func` with arguments `args`. This is for a single task execution and blocks until the function is complete.

- **Non-Blocking (Asynchronous) Methods:** These return immediately, allowing the main program to do other work. The results are obtained later.
    - `pool.map_async(func, iterable)`: The async version of `map`. Returns an `AsyncResult` object.
    - `pool.apply_async(func, args)`: The async version of `apply`. Returns an `AsyncResult` object.

To get the result from an `AsyncResult` object (`res`), you call `res.get()`.

**Example: CPU-Bound Task with `Pool.map`**

Let's simulate a heavy computation.

In [ ]:
import multiprocessing
import time
import os

def square(x):
    """A CPU-intensive function."""
    # Simulate work
    # time.sleep(0.01)
    return x * x

# Note: In a script, this would be inside the if __name__ == '__main__': block

# Use all available CPU cores
pool = multiprocessing.Pool(processes=os.cpu_count())

inputs = range(20)

start_time = time.time()

# map blocks until all results are ready
results = pool.map(square, inputs)

# Always close and join a pool
pool.close() # Prevents any more tasks from being submitted
pool.join()  # Waits for the worker processes to exit

end_time = time.time()

print(f"Results: {results}")
print(f"Time taken: {end_time - start_time:.4f} seconds")

## 5. Advanced Topics and Best Practices

- **Choosing Chunksize in `pool.map`:** For very long iterables, specifying a `chunksize` can significantly improve performance by reducing the overhead of sending small chunks of data to worker processes. A good starting point is `chunksize = len(iterable) // (len(pool) * 4)`.

- **Process vs. `Pool`:** Use `Process` for long-running, heterogeneous tasks (e.g., a producer and a consumer). Use `Pool` for homogeneous, embarrassingly parallel tasks where you need to apply the same function to many different pieces of data.

- **Overhead:** Creating processes is not free. There's a performance cost for process creation, data pickling/unpickling (serialization), and IPC. Multiprocessing is only beneficial if the task's computation time is significantly greater than this overhead.

- **Daemon Processes:** A process can be flagged as a daemon process (`p.daemon = True`). Daemon processes are terminated automatically when the main program exits. They are useful for background tasks but cannot create new processes themselves.

- **Exception Handling in Pools:** If a worker in a `Pool` raises an exception, that exception will be re-raised in the main process when you try to retrieve the result with `.get()`. It's crucial to wrap your result-retrieval calls in `try...except` blocks.